In [ ]:
import holoviews as hv
hv.extension("bokeh")
import hvplot.pandas
import pandas as pd
from pathlib import Path
import sys
sys.path.append("/proj/gaia-climate/team/kirill/gaia-surrogate")
from gaia.training import load_hparams_file


In [ ]:
def get_metrics(model_dir, dataset = "spcam"):
    metrics = pd.read_json(next(Path(model_dir).glob(f"*{dataset}*"))).T.squeeze()
    hparams = load_hparams_file(model_dir)
    metrics["subsample"] = int(hparams["dataset_params"]["train"].get("subsample",1))
    metrics["is_finetuned"]  = int(hparams.get("is_finetuned",False))
    metrics["base_model"] = "random"
    metrics["samples"] = 24 * 96 * 144 * 54 // metrics["subsample"]
    
    if "cam4" in str(model_dir):
        return pd.Series()
        # metrics["base_model"] = "cam4"
        # metrics["is_finetuned"] = True
        # metrics["subsample"] = 
    
    if metrics["is_finetuned"]:
        metrics["base_model"] = "cam4"        
        
        
    return metrics

data = pd.concat([get_metrics(f).to_frame().T for f in Path("lightning_logs").glob("*")]).dropna()
data = data.iloc[:,1:]
data.columns = [c.replace("test_skill_ave_trunc_","") for c in data.columns]

data["PTEQ"] = data.loc[:,[c for c in data.columns if "PTEQ_" in c]].iloc[:,8:].mean(1)
data["PTTEND"] = data.loc[:,[c for c in data.columns if "PTTEND_" in c]].iloc[:,8:].mean(1)




In [ ]:
metrics = "PRECT,PRECC,PTTEND,PTEQ".split(",")

def plot_one(metric):
    temp = data.sort_values(metric)
    return temp.hvplot.line(x = "samples", y = metric, groupby = "base_model", logx = True, line_width = .5, width = 400, grid = True)*\
    temp.hvplot.scatter(x = "samples", y = metric, groupby = "base_model", logx = True, padding = .1)

plots = hv.Layout([plot_one(metric).overlay("base_model").opts(legend_position = "bottom_right", title=f"skill {metric}") for metric in metrics])
plots = plots.cols(2)
plots

In [ ]:
hv.save(plots, "fine_tune.html")